overview of this notebook...

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import xskillscore as xs
%load_ext blackcellmagic

Use the same data as in 01_Deterministic.py

In [8]:
stores = np.arange(4)
skus = np.arange(3)
dates = pd.date_range("1/1/2020", "1/5/2020", freq="D")

rows = []
for _, date in enumerate(dates):
    for _, store in enumerate(stores):
        for _, sku in enumerate(skus):
            rows.append(
                dict(
                    {
                        "DATE": date,
                        "STORE": store,
                        "SKU": sku,
                        "QUANTITY_SOLD": np.random.randint(10),
                    }
                )
            )
df = pd.DataFrame(rows)
df.rename(columns={"QUANTITY_SOLD": "y"}, inplace=True)
df.set_index(['DATE', 'STORE', 'SKU'], inplace=True)
df.head()

y
DATE       STORE SKU   
2020-01-01 0     0    4
                 1    1
                 2    0
           1     0    1
                 1    8

Make multiple predictions...

Append the predictions to the df using a `member` column...

In [14]:
tmp = df.copy()
for i in range(1, 7):
    tmp['member'] = i
    noise = np.random.uniform(-1, 1, size=len(df['y']))
    tmp['yhat'] = (df['y'] + (df['y'] * noise)).astype(int)
    if i == 1:
        df_yhat = tmp.copy()
    else:
        df_yhat = df_yhat.append(tmp)
df_yhat

y  member  yhat
DATE       STORE SKU                 
2020-01-01 0     0    4       1     5
                 1    1       1     0
                 2    0       1     0
           1     0    1       1     1
                 1    8       1    15
...                  ..     ...   ...
2020-01-05 2     1    5       6     9
                 2    1       6     0
           3     0    3       6     2
                 1    1       6     1
                 2    0       6     0

[360 rows x 3 columns]

Drop the `y` column and add `member` to the index...

In [15]:
df_yhat.drop('y', axis=1, inplace=True)
df_yhat.set_index(['member'], append=True, inplace=True)
df_yhat

yhat
DATE       STORE SKU member      
2020-01-01 0     0   1          5
                 1   1          0
                 2   1          0
           1     0   1          1
                 1   1         15
...                           ...
2020-01-05 2     1   6          9
                 2   6          0
           3     0   6          2
                 1   6          1
                 2   6          0

[360 rows x 1 columns]

Convert the target dataframe to an xarray.Dataset... 

In [16]:
ds = df.to_xarray()
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
Data variables:
    y        (DATE, STORE, SKU) int64 4 1 0 1 8 8 4 0 4 6 ... 9 0 2 9 5 1 3 1 0

Add the predicted dataframe as an `xarray.DataArray` to the `xarray.Dataset`

In [17]:
ds['yhat'] = df_yhat.to_xarray()['yhat']
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4, member: 6)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
  * member   (member) int64 1 2 3 4 5 6
Data variables:
    y        (DATE, STORE, SKU) int64 4 1 0 1 8 8 4 0 4 6 ... 9 0 2 9 5 1 3 1 0
    yhat     (DATE, STORE, SKU, member) int64 5 0 7 3 2 0 0 1 ... 1 0 0 0 0 0 0

# Using xskillscore - CRPS

What is CRPS...

In [ ]:
It's useage om 

Apply CRPS over the member dimension... This will return the CRPS as a date, store and sku level

In [23]:
ds.xs.crps_ensemble('y', 'yhat', dim='member')

<xarray.DataArray (DATE: 5, STORE: 4, SKU: 3)>
array([[[1.08333333, 0.02777778, 0.        ],
        [0.11111111, 2.47222222, 1.75      ],
        [0.88888889, 0.        , 0.83333333],
        [2.36111111, 1.08333333, 0.        ]],

       [[0.        , 0.11111111, 1.91666667],
        [1.52777778, 0.19444444, 2.72222222],
        [1.19444444, 0.36111111, 2.11111111],
        [1.08333333, 2.38888889, 0.61111111]],

       [[0.02777778, 0.        , 1.13888889],
        [0.        , 2.30555556, 0.94444444],
        [0.        , 0.63888889, 1.19444444],
        [0.80555556, 0.63888889, 2.30555556]],

       [[0.44444444, 0.94444444, 0.58333333],
        [1.41666667, 0.41666667, 0.88888889],
        [2.11111111, 1.33333333, 0.97222222],
        [0.        , 1.97222222, 0.        ]],

       [[1.58333333, 1.61111111, 0.        ],
        [3.33333333, 0.        , 0.94444444],
        [2.91666667, 1.72222222, 0.44444444],
        [0.27777778, 0.11111111, 0.        ]]])
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2

To return an overal CRPS it is recommened averaging over all dimensions before using `crps`...

In [31]:
y = ds['y'].mean(dim=['DATE', 'STORE', 'SKU'])
print(y)
yhat = ds['yhat'].mean(dim=['DATE', 'STORE', 'SKU'])
print(yhat)
xs.crps_ensemble(y, yhat, dim='member')

<xarray.DataArray 'y' ()>
array(3.98333333)
<xarray.DataArray 'yhat' (member: 6)>
array([3.45      , 3.11666667, 3.26666667, 3.93333333, 3.75      ,
       3.4       ])
Coordinates:
  * member   (member) int64 1 2 3 4 5 6


<xarray.DataArray ()>
array(0.34212963)